In [2]:
import tensorflow as tf
import os

print(tf.__version__)

2.0.0


In [3]:
# 自己实现一个卷积运算函数,输入时初始数组和卷积核数组,输出卷积计算后的结果

def conv2d(X, K):
    h, w = K.shape
    # 先计算出输出结果 y 的形状，构造y，方便计算
    Y = tf.Variable(tf.zeros((X.shape[0] - h + 1, X.shape[1] - w +1)))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            # 将 y 的 i,j 赋值成 和
            Y[i,j].assign(tf.cast(tf.reduce_sum(X[i:i+h, j:j+w] * K), dtype=tf.float32))
    return Y


In [5]:
X = tf.constant([[0,1,2], [3,4,5], [6,7,8]])
K = tf.constant([[0,1], [2,3]])
conv2d(X, K)

<tf.Variable 'Variable:0' shape=(2, 2) dtype=float32, numpy=
array([[19., 25.],
       [37., 43.]], dtype=float32)>

In [8]:
# 构造卷积层， conv2d + bias

class Conv2D(tf.keras.layers.Layer):
    def __init__(self):
        super().__init__()
    
    # 由于有 训练的参数，所以使用 build构建
    def build(self, kernal_size):
        self.w = self.add_weight(name='weight',shape=kernal_size,initializer=tf.random_normal_initializer())
        self.b = self.add_weight(name='bias',shape=kernal_size,initializer=tf.zeros_initializer())
    def call(self, inputs):
        y = conv2d(inputs, self.w) + self.b
        
        return y

In [9]:
# 构建数据
X = tf.Variable(tf.ones((6,8)))
X[:, 2:6].assign(tf.zeros(X[:,2:6].shape))


<tf.Variable 'UnreadVariable' shape=(6, 8) dtype=float32, numpy=
array([[1., 1., 0., 0., 0., 0., 1., 1.],
       [1., 1., 0., 0., 0., 0., 1., 1.],
       [1., 1., 0., 0., 0., 0., 1., 1.],
       [1., 1., 0., 0., 0., 0., 1., 1.],
       [1., 1., 0., 0., 0., 0., 1., 1.],
       [1., 1., 0., 0., 0., 0., 1., 1.]], dtype=float32)>

In [13]:
K = tf.constant([[1,-1]], dtype = tf.float32)
Y = conv2d(X, K)
Y

<tf.Variable 'Variable:0' shape=(6, 7) dtype=float32, numpy=
array([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
       [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
       [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
       [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
       [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
       [ 0.,  1.,  0.,  0.,  0., -1.,  0.]], dtype=float32)>

In [15]:
X = tf.reshape(X, (1, 6, 8, 1))
Y = tf.reshape(Y, (1,6,7,1))

conv2d = tf.keras.layers.Conv2D(1, (1,2))

In [18]:
Y_hat = conv2d(X)
for i in range(20):
    with tf.GradientTape(watch_accessed_variables=False) as g:
        g.watch(conv2d.weights[0])
        Y_hat = conv2d(X)
        l = (abs(Y_hat - Y)) ** 2
        dl = g.gradient(l, conv2d.weights[0])
        lr = 3e-2
        update = tf.multiply(lr, dl)
        updated_weights = conv2d.get_weights()
        updated_weights[0] = conv2d.weights[0] - update
        conv2d.set_weights(updated_weights)  

        if (i + 1)% 2 == 0:
            print('batch %d, loss %.3f' % (i + 1, tf.reduce_sum(l)))


batch 2, loss 0.210
batch 4, loss 0.086
batch 6, loss 0.035
batch 8, loss 0.014
batch 10, loss 0.006
batch 12, loss 0.002
batch 14, loss 0.001
batch 16, loss 0.000
batch 18, loss 0.000
batch 20, loss 0.000


In [19]:
tf.reshape(conv2d.get_weights()[0],(1,2))


<tf.Tensor: id=3352, shape=(1, 2), dtype=float32, numpy=array([[ 0.9991463, -1.0008522]], dtype=float32)>